# core

> Fill in a module description here

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import requests
import time
import os
import subprocess
from tqdm import tqdm
from glob import glob
from datetime import datetime
import xml.etree.ElementTree as ET
from xml.dom import minidom

In [ ]:
#| export
class CoreClient:
    def __init__(self):
        self.manifest_data = None
        self.canvases = None

    @staticmethod
    def main(manifest_url, output_dir, src_path, verbose=False, resized_width=1200, interval=1):
        """メイン実行メソッド"""
        client = CoreClient()
        paths = client._setup_directories(output_dir)
        
        client.download_images(
            manifest_url=manifest_url,
            output_dir=paths['img_dir'],
            verbose=verbose,
            resized_width=resized_width,
            interval=interval
        )
       
        client.run_ocr(
            input_dir=paths['img_dir'],
            output_dir=paths['ocr_dir'],
            src_path=src_path
        )
        
        client.create_tei_xml(
            manifest_url=manifest_url,
            input_dir=paths['ocr_dir'],
            output_path=paths['xml_path']
        )

    def _setup_directories(self, item_dir):
        """ディレクトリ構造のセットアップ"""
        paths = {
            'img_dir': f"{item_dir}/img",
            'ocr_dir': f"{item_dir}/ocr",
            'xml_path': f"{item_dir}/{item_dir.split('/')[-1]}.xml"
        }
        
        for dir_path in [paths['img_dir'], paths['ocr_dir']]:
            os.makedirs(dir_path, exist_ok=True)
            
        return paths

    def _fetch_manifest(self, manifest_url):
        """マニフェストデータの取得"""
        self.manifest_data = requests.get(manifest_url).json()
        self.canvases = self.manifest_data["sequences"][0]["canvases"]
        return self.manifest_data, self.canvases

    def download_images(self, manifest_url, output_dir, interval=1, verbose=False, resized_width=1200):
        """画像のダウンロード"""
        _, canvases = self._fetch_manifest(manifest_url)
        
        for i in tqdm(range(len(canvases)), desc="Downloading images"):
            self._download_single_image(
                canvas=canvases[i],
                index=i,
                output_dir=output_dir,
                resized_width=resized_width,
                verbose=verbose,
                interval=interval
            )

    def _download_single_image(self, canvas, index, output_dir, resized_width, verbose, interval):
        """単一画像のダウンロード"""
        image_url = canvas["images"][0]["resource"]["@id"].replace(
            "/full/full/",
            f"/full/{resized_width},/"
        )
        output_path = f"{output_dir}/{str(index + 1).zfill(4)}.jpg"
        
        if os.path.exists(output_path):
            return
            
        if verbose:
            print({"image_url": image_url, "output_path": output_path})
            
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        
        response = requests.get(image_url)
        with open(output_path, "wb") as f:
            f.write(response.content)
            
        time.sleep(interval)

    def run_ocr(self, input_dir, output_dir, src_path):
        """OCR処理の実行"""
        current_path = os.getcwd()

        input_abs_path = os.path.abspath(input_dir)
        output_abs_path = os.path.abspath(output_dir)

        try:
            os.chdir(src_path)
            
            subprocess.run([
                "python",
                "ocr.py",
                "--sourcedir", input_abs_path,
                "--output", output_abs_path
            ])
        finally:
            os.chdir(current_path)

    def create_tei_xml(self, manifest_url, input_dir, output_path):
        """TEI/XMLの作成"""
        if not self.manifest_data:
            self._fetch_manifest(manifest_url)
            
        files = sorted(glob(f"{input_dir}/*.xml"))
        
        # TEI文書の作成
        tei = self._create_tei_base(manifest_url)
        body = tei.find(".//body")
        facsimile = tei.find(".//facsimile")
        
        # ページごとの処理
        for i, file_path in tqdm(enumerate(files), desc="Creating TEI/XML"):
            self._process_page(i, file_path, body, facsimile)
            
        # XML出力
        self._write_tei_xml(tei, output_path)

    def _create_tei_base(self, manifest_url):
        """TEI基本構造の作成"""
        tei = ET.Element("TEI", xmlns="http://www.tei-c.org/ns/1.0")
        
        # ヘッダー作成
        header = self._create_header(manifest_url, tei)
        
        # テキスト要素作成
        text = ET.SubElement(tei, "text")
        ET.SubElement(text, "body")
        
        # facsimile要素作成
        facsimile = ET.SubElement(tei, "facsimile")
        facsimile.set("sameAs", manifest_url)
        
        return tei

    def _create_header(self, manifest_url, tei):
        """TEIヘッダーの作成"""
        header = ET.SubElement(tei, "teiHeader")
        file_desc = ET.SubElement(header, "fileDesc")
        
        # titleStmt
        title_stmt = ET.SubElement(file_desc, "titleStmt")
        title = ET.SubElement(title_stmt, "title")
        title.text = f"OCR結果: {manifest_url}"
        
        # publicationStmt
        pub_stmt = ET.SubElement(file_desc, "publicationStmt")
        publisher = ET.SubElement(pub_stmt, "publisher")
        publisher.text = "NDL古典OCR"
        date = ET.SubElement(pub_stmt, "date")
        date.text = datetime.now().strftime("%Y-%m-%d")
        
        # sourceDesc
        source_desc = ET.SubElement(file_desc, "sourceDesc")
        bibl = ET.SubElement(source_desc, "bibl")
        ptr = ET.SubElement(bibl, "ptr")
        ptr.set("target", manifest_url)
        
        return header
    
    def _create_surface(self, canvas, facsimile):
        """surface要素の作成"""
        surface = ET.SubElement(facsimile, "surface")
        
        # 基本属性の設定
        surface.set("sameAs", canvas["@id"])
        surface.set("ulx", "0")
        surface.set("uly", "0")
        surface.set("lrx", str(canvas["images"][0]["resource"]["width"]))
        surface.set("lry", str(canvas["images"][0]["resource"]["height"]))
        
        # ラベルの追加
        label = ET.SubElement(surface, "label")
        label.text = canvas["label"]
        
        # graphic要素の追加
        graphic = ET.SubElement(surface, "graphic")
        graphic.set("url", canvas["images"][0]["resource"]["@id"])
        
        # service情報がある場合は追加
        if "service" in canvas["images"][0]["resource"]:
            graphic.set("sameAs", canvas["images"][0]["resource"]["service"]["@id"])
            graphic.set("width", str(canvas["images"][0]["resource"]["width"]) + "px")
            graphic.set("height", str(canvas["images"][0]["resource"]["height"]) + "px")
        
        return surface

    def _create_page_div(self, body, index, canvas):
        """ページのdiv要素作成"""
        div = ET.SubElement(body, "ab")
        div.set("n", str(index + 1))
        div.set("type", "page")
        div.set("facs", canvas["images"][0]["resource"]["@id"])
        return div

    def _process_lines(self, page_xml, div, surface, canvas_width):
        """行単位の処理"""
        page_info = page_xml.find(".//PAGE")
        ocr_page_width = int(page_info.get("WIDTH", "0"))
        
        for line in page_xml.findall(".//LINE"):
            # 座標の計算
            line_x = int(line.get("X", "0"))
            line_y = int(line.get("Y", "0"))
            line_width = int(line.get("WIDTH", "0"))
            line_height = int(line.get("HEIGHT", "0"))
            
            # スケール調整
            ratio = canvas_width / ocr_page_width
            fixed_x = int(line_x * ratio)
            fixed_y = int(line_y * ratio)
            fixed_width = int(line_width * ratio)
            fixed_height = int(line_height * ratio)
            
            # line_indexの設定
            line_index = str(int(line.get("ORDER", "")) + 1)
            
            # zone要素の作成
            zone = self._create_zone(surface, line, fixed_x, fixed_y, 
                                fixed_width, fixed_height, line_index)
            
            # テキスト要素の追加
            self._add_text_elements(div, line, line_index, zone)

    def _create_zone(self, surface, line, x, y, width, height, line_index):
        """zone要素の作成"""
        zone = ET.SubElement(surface, "zone")
        zone.set("type", line.get("TYPE", ""))
        zone.set("n", line_index)
        zone.set("ulx", str(x))
        zone.set("uly", str(y))
        zone.set("lrx", str(x + width))
        zone.set("lry", str(y + height))
        zone.set("xml:id", f"zone-{line_index}")
        return zone

    def _add_text_elements(self, div, line, line_index, zone):
        """テキスト要素の追加"""
        # 改行要素
        ET.SubElement(div, "lb")
        
        # セグメント要素
        seg = ET.SubElement(div, "seg")
        seg.set("type", line.get("TYPE", ""))
        seg.set("n", line_index)
        seg.set("corresp", f"#{zone.get('xml:id')}")
        seg.text = line.get("STRING", "")

    def _process_page(self, index, file_path, body, facsimile):
        """ページ単位の処理"""
        canvas = self.canvases[index]
        canvas_width = canvas["images"][0]["resource"]["width"]
        
        # surface要素の作成
        surface = self._create_surface(canvas, facsimile)
        
        # OCRデータの処理
        with open(file_path, "r", encoding="utf-8") as f:
            page_xml = ET.fromstring(f.read())
            
        div = self._create_page_div(body, index, canvas)
        self._process_lines(page_xml, div, surface, canvas_width)

    def _write_tei_xml(self, tei, output_path):
        """TEI/XMLファイルの書き出し"""
        rough_string = ET.tostring(tei, 'utf-8')
        reparsed = minidom.parseString(rough_string)
        xml_content = reparsed.toprettyxml(indent="  ")
        
        final_xml = f'''<?xml version="1.0" encoding="UTF-8"?>
<?xml-model href="http://www.tei-c.org/release/xml/tei/custom/schema/relaxng/tei_all.rng" type="application/xml" schematypens="http://relaxng.org/ns/structure/1.0"?>
<?xml-model href="http://www.tei-c.org/release/xml/tei/custom/schema/relaxng/tei_all.rng" type="application/xml" schematypens="http://purl.oclc.org/dsdl/schematron"?>
{xml_content[xml_content.find('<TEI'):]}'''

        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        with open(output_path, 'w', encoding='utf-8') as f:
            f.write(final_xml)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()